In [1]:
#import face_recognition as fr
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import pandas as pd

In [6]:
paths = ["data/all/train/","data/all/test/"] #["data/CK+/","data/fer/train/"]
# data  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs = []
state = []

In [7]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state.extend([item for p in listdir(f"{path}{item}")])

In [8]:
len(state),len(imgs)

(50964, 50964)

In [9]:
imgs[:5],state[:5]

(['data/all/train/surprise/Training_86715155.jpg',
  'data/all/train/surprise/Training_63289181.jpg',
  'data/all/train/surprise/Training_85075254.jpg',
  'data/all/train/surprise/Training_2929493.jpg',
  'data/all/train/surprise/Abdullah_Nasseef_0001.jpg'],
 ['surprise', 'surprise', 'surprise', 'surprise', 'surprise'])

In [10]:
errs = []
try:
    imgs_ = []
    for p in imgs:
        temp = Image.open(p)
        save = temp.copy()
        imgs_.append(save)
        temp.close()
except:
    errs.append(p)

In [11]:
errs

[]

In [12]:
len(imgs)


50964

In [13]:
imgs_array = []
HEIGHT, WIDTH =48,48
for f in imgs_:
    img = f.convert("L").resize((HEIGHT, WIDTH))
    imgs_array.append(np.array(img))

In [14]:
imgs_array = [el/255 for el in imgs_array]


In [15]:
imgs_array = np.array(imgs_array)

In [16]:
imgs_array.shape

(50964, 48, 48)

In [25]:
#imgs_array = imgs_array.reshape((50964,48,48,1))

In [17]:
categories = ['sadness', 'happiness', 'anger', 'disgust', 'neutral', 'surprise', 'fear']

In [18]:
categories

['sadness', 'happiness', 'anger', 'disgust', 'neutral', 'surprise', 'fear']

In [19]:
y=pd.DataFrame(state)
y_dummies = pd.get_dummies(y)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_array,y_dummies)

In [21]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD, Adamax
import keras

In [22]:
opt = Adam(learning_rate=0.001)

In [23]:
model = Sequential()
#model.add(Flatten())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(imgs_array.shape[1:])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=512, kernel_size=3, activation='relu', input_shape=(imgs_array.shape[1:])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])

2021-11-15 15:21:40.654902: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2021-11-15 15:21:40.654939: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: oobonioo-X570-AORUS-ELITE
2021-11-15 15:21:40.654947: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: oobonioo-X570-AORUS-ELITE
2021-11-15 15:21:40.655062: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.91.3
2021-11-15 15:21:40.655085: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2021-11-15 15:21:40.655091: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.91.3
2021-11-15 15:21:40.655348: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instr

In [24]:
from keras.models import Sequential, load_model, model_from_json
from keras import callbacks, optimizers
import tensorflow as tf
from datetime import date

fecha=str(date.today().year)+str(date.today().month)+str(date.today().day)    
symbol = 'alldata'
h5 = symbol + '_1_' + fecha + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='loss',
                                       verbose=0,
                                       save_best_only=True,
                                       #save_weights_only=True,
                                       mode='auto',
                                       save_freq=1)
callback = [checkpoint]
json = symbol + '_best_model' + fecha + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)

modelo = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 5000,callbacks = callback,validation_split = 0.1)

2021-11-15 15:22:30.682121: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5000
1075/1075 [==============================] - 99s 91ms/step - loss: 1.7416 - accuracy: 0.3398 - val_loss: 1.6291 - val_accuracy: 0.3761
Epoch 2/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 1.5056 - accuracy: 0.4415 - val_loss: 1.4227 - val_accuracy: 0.4711
Epoch 3/5000
1075/1075 [==============================] - 13s 12ms/step - loss: 1.4465 - accuracy: 0.4663 - val_loss: 1.4282 - val_accuracy: 0.4698
Epoch 4/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 1.4189 - accuracy: 0.4762 - val_loss: 1.4278 - val_accuracy: 0.4653
Epoch 5/5000
1075/1075 [==============================] - 11s 11ms/step - loss: 1.3995 - accuracy: 0.4827 - val_loss: 1.3463 - val_accuracy: 0.5025
Epoch 6/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 1.3876 - accuracy: 0.4859 - val_loss: 1.3277 - val_accuracy: 0.5111
Epoch 7/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 1.3636 - accuracy: 0.4969 - val_

1075/1075 [==============================] - 11s 10ms/step - loss: 0.9863 - accuracy: 0.6400 - val_loss: 1.1765 - val_accuracy: 0.5844
Epoch 57/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.9949 - accuracy: 0.6346 - val_loss: 1.1101 - val_accuracy: 0.6027
Epoch 58/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.9763 - accuracy: 0.6452 - val_loss: 1.0960 - val_accuracy: 0.6045
Epoch 59/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.9773 - accuracy: 0.6428 - val_loss: 1.1581 - val_accuracy: 0.5888
Epoch 60/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.9802 - accuracy: 0.6405 - val_loss: 1.1034 - val_accuracy: 0.6035
Epoch 61/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.9639 - accuracy: 0.6522 - val_loss: 1.1270 - val_accuracy: 0.6019
Epoch 62/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.9589 - accuracy: 0.6511 - val_loss: 1

1075/1075 [==============================] - 11s 10ms/step - loss: 0.8342 - accuracy: 0.6983 - val_loss: 1.1324 - val_accuracy: 0.6074
Epoch 112/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.8315 - accuracy: 0.6986 - val_loss: 1.1403 - val_accuracy: 0.6121
Epoch 113/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.8268 - accuracy: 0.7005 - val_loss: 1.1400 - val_accuracy: 0.6100
Epoch 114/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.8286 - accuracy: 0.6979 - val_loss: 1.2963 - val_accuracy: 0.5650
Epoch 115/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.8343 - accuracy: 0.6977 - val_loss: 1.1450 - val_accuracy: 0.6037
Epoch 116/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.8248 - accuracy: 0.6993 - val_loss: 1.1509 - val_accuracy: 0.6116
Epoch 117/5000
1075/1075 [==============================] - 11s 10ms/step - loss: 0.8214 - accuracy: 0.7023 - val_l

KeyboardInterrupt: 

In [25]:
paths = ["data/predict/"]
# data_pred  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs_pred = []
state_pred = []

In [26]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs_pred.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state_pred.extend([item for p in listdir(f"{path}{item}")])



In [27]:
imgs_pred

['data/predict/surprise/1636579058672.jpg',
 'data/predict/surprise/1636579058703.jpg',
 'data/predict/surprise/1636579058689.jpg',
 'data/predict/anger/1636579058658.jpg',
 'data/predict/anger/1636579058692.jpg',
 'data/predict/neutral/neutral_2.jpg',
 'data/predict/neutral/neutral_3.jpg',
 'data/predict/neutral/1636579058676.jpg',
 'data/predict/neutral/neutral_0.jpg',
 'data/predict/neutral/1636579058682.jpg',
 'data/predict/neutral/1636579058712.jpg',
 'data/predict/neutral/neutral_1.jpg',
 'data/predict/neutral/neutral_4.jpg',
 'data/predict/happiness/happy_6.png',
 'data/predict/happiness/happy_8.png',
 'data/predict/happiness/1636579058696.jpg',
 'data/predict/happiness/happy_4.png',
 'data/predict/happiness/happy_10.png',
 'data/predict/happiness/happy_11.png',
 'data/predict/happiness/1636579058663.jpg',
 'data/predict/happiness/1636579058717.jpg',
 'data/predict/happiness/happy_3.jpeg',
 'data/predict/happiness/happy_14.jpg',
 'data/predict/happiness/happy_9.png',
 'data/pred

In [30]:
imgs_pred_ = []
for p in imgs_pred:
    temp = Image.open(p)
    save = temp.copy()
    imgs_pred_.append(save)
    temp.close()

imgs_array_pred = []
for f in imgs_pred_:
    img = f.convert("L").resize((HEIGHT, WIDTH))
    imgs_array_pred.append(np.array(img))
imgs_array_pred = np.array(imgs_array_pred)

In [31]:
prediction = model.predict(imgs_array_pred)

In [32]:
len(prediction)

34

In [33]:
prediction[:5]

array([[0.0000000e+00, 0.0000000e+00, 5.2877351e-30, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [9.9999988e-01, 0.0000000e+00, 6.0679409e-17, 0.0000000e+00,
        7.0373964e-08, 0.0000000e+00, 0.0000000e+00],
       [1.2767212e-26, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.6539792e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        9.8346025e-01, 0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [34]:
prediction.argmax(axis = -1)


array([4, 0, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [35]:
cats= y_dummies.columns
cats = [x.replace("0_","") for x in cats]

In [36]:
cats

['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

In [37]:
states_model = []
for i in range(len(prediction)):
    states_model.append(cats[prediction[i].argmax()])
    print(state_pred[i],cats[prediction[i].argmax()],imgs_pred[i])

surprise neutral data/predict/surprise/1636579058672.jpg
surprise anger data/predict/surprise/1636579058703.jpg
surprise neutral data/predict/surprise/1636579058689.jpg
anger neutral data/predict/anger/1636579058658.jpg
anger neutral data/predict/anger/1636579058692.jpg
neutral neutral data/predict/neutral/neutral_2.jpg
neutral neutral data/predict/neutral/neutral_3.jpg
neutral fear data/predict/neutral/1636579058676.jpg
neutral neutral data/predict/neutral/neutral_0.jpg
neutral neutral data/predict/neutral/1636579058682.jpg
neutral neutral data/predict/neutral/1636579058712.jpg
neutral neutral data/predict/neutral/neutral_1.jpg
neutral neutral data/predict/neutral/neutral_4.jpg
happiness neutral data/predict/happiness/happy_6.png
happiness neutral data/predict/happiness/happy_8.png
happiness neutral data/predict/happiness/1636579058696.jpg
happiness neutral data/predict/happiness/happy_4.png
happiness neutral data/predict/happiness/happy_10.png
happiness neutral data/predict/happiness

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(state_pred,states_model)

0.20588235294117646